# Poisson Equation in non-cartesian domains

In [1]:
import underworld3 as uw
from mpi4py import MPI ## library for displaying everything

We define a mesh on a 2-dimensional annulus. With an inner radius of 0.1, an outer radius of 1, a mesh cell size of 0.1 and a quadrature degree of 5. We solve $\nabla \cdot ((1 + \phi) \nabla \phi) = 4 + 8 x^2 + 8 y^2$ with boundary conditions of $\phi=0.1$ on the inner boundary and $\phi=1$ on the outer boundary.

In [2]:
mesh = uw.meshing.Annulus(radiusInner=0.1, radiusOuter=1, cellSize=0.025, qdegree=5)

Lets print out the mesh

In [3]:
## print out what the mesh looks like
if uw.mpi.size == 1:

    # plot the mesh
    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [750, 750]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("tmp_mesh.vtk")
    pvmesh = pv.read("tmp_mesh.vtk")

    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        use_transparency=False,
    )

    pl.show(cpos="xy")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_39276/1705306568.py:12: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, height=750, min_width=750, sizing_mode='stretch_width')

our mesh still has cartesian coordinates x and y

In [4]:
mesh.X

Matrix([[N.x, N.y]])

We define our frield $\phi$ on the mesh as a scalar with degree 3.

In [5]:
phi = uw.discretisation.MeshVariable("r\phi", mesh, 1, degree=3)

Create a poisson solver

In [6]:
poisson_solver = uw.systems.Poisson(mesh, u_Field=phi)
poisson_solver.constitutive_model = uw.systems.constitutive_models.DiffusionModel(mesh.dim)

Set the source term $S(\vec{x}) = 4 + 8 x^2 + 8 y^2$

In [7]:
x, y = mesh.X ## get the symbolic repressentations of the coordiantes. Note these are cartesian coordinates
source_fn = 4 + 8 * x**2 + 8 * y**2
poisson_solver.f = -source_fn

Define the symbolic repressentation of the diffusivity

In [8]:
k_fn = 1 + phi.sym[0]

Set the boundary conditions. We set the bottom and top boundary to 0.01 and 1 respectively.

In [9]:
poisson_solver.add_dirichlet_bc((0.01,), "Lower", (0,))
poisson_solver.add_dirichlet_bc((1,), "Upper", (0,))

Now, we will write the diffusivity constant $k = \alpha + (1 - \alpha) k_{fn}$ and slowly increase the non-linearity ($\alpha$), using our previous soution to $\phi$ as our guess.

In [10]:
## get our initial guess by solving without any non-linearity (a = 1)
poisson_solver.constitutive_model.Parameters.diffusivity = 1
poisson_solver.solve()


aList = [1 - da/10 for da in range(11)]
for a in aList:
    print(a)
    poisson_solver.constitutive_model.Parameters.diffusivity = a + (1 - a) * k_fn
    poisson_solver.solve(zero_init_guess=False)

  0 SNES Function norm 58.1252 
  1 SNES Function norm 0.00971717 
  2 SNES Function norm 1.22987e-07 
Nonlinear Poisson_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2
1.0
  0 SNES Function norm 1.22987e-07 
  1 SNES Function norm < 1.e-11
Nonlinear Poisson_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
0.9
  0 SNES Function norm 0.0167673 
  1 SNES Function norm 6.11828e-05 
  2 SNES Function norm 4.46473e-08 
Nonlinear Poisson_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2
0.8
  0 SNES Function norm 0.0147949 
  1 SNES Function norm 8.08219e-05 
  2 SNES Function norm 9.78251e-08 
Nonlinear Poisson_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2
0.7
  0 SNES Function norm 0.0132215 
  1 SNES Function norm 8.57258e-05 
  2 SNES Function norm 9.14622e-08 
Nonlinear Poisson_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2
0.6
  0 SNES Function norm 0.0119415 
  1 SNES Function norm 9.26836e-05 
  2 SNES Fu

The analytic solution is $\phi = x^2 + y^2$, so lets plot the numerical, analytic and difference between the two.

In [11]:
x,y = mesh.X ## access the symbolic repressentations of the coordinates
analytic_fn = x**2 + y**2 ## the analytic solution

with mesh.access():    
    mesh_analytic_soln  = uw.function.evaluate(analytic_fn, mesh.data, mesh.N)
    mesh_numerical_soln = uw.function.evaluate(poisson_solver.u.fn, mesh.data, mesh.N)

In [12]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("mesh_tmp.vtk")
    pvmesh = pv.read("mesh_tmp.vtk")

    with mesh.access():
        pvmesh.point_data["2DannulusAnalytic"] = mesh_analytic_soln

    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="2DannulusAnalytic",
        use_transparency=False,
        opacity=0.5,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    # pl.screenshot(filename="test.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_39276/1977401067.py:10: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='c95...], height=500, min_width=500, sizing_mode='stretch_width')

In [13]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("mesh_tmp.vtk")
    pvmesh = pv.read("mesh_tmp.vtk")

    with mesh.access():
        pvmesh.point_data["2DannulusNumerical"] = mesh_numerical_soln

    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="2DannulusNumerical",
        use_transparency=False,
        opacity=0.5,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    # pl.screenshot(filename="test.png")


/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_39276/3384764692.py:10: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='0f9...], height=500, min_width=500, sizing_mode='stretch_width')

In [14]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("mesh_tmp.vtk")
    pvmesh = pv.read("mesh_tmp.vtk")

    with mesh.access():
        pvmesh.point_data["2DannulusDiff"] = mesh_analytic_soln - mesh_numerical_soln

    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="2DannulusDiff",
        use_transparency=False,
        opacity=0.5,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    # pl.screenshot(filename="test.png")


/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_39276/1609776605.py:10: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='e78...], height=500, min_width=500, sizing_mode='stretch_width')

Now, we do have an error in our solution, however the error here is very small. The difference is less than $1 \%$ between the analytic and numerical results.

In [15]:
with mesh.access():
    maxDiff = float('-inf')
    for index in range(len(mesh_analytic_soln)):
    
        if ( abs(mesh_analytic_soln[index]-mesh_numerical_soln[index])/mesh_analytic_soln[index] > maxDiff ):
            maxDiff = abs(mesh_analytic_soln[index]-mesh_numerical_soln[index])/mesh_analytic_soln[index]
    print("Maximum percentage difference between analytic and numerical solution is", maxDiff*100)
            

Maximum percentage difference between analytic and numerical solution is 0.7473005038999556


Now, lets solve a similar equation, $\nabla \cdot ((1 + \phi) \nabla \phi) = 6 + 10 x^2 + 10 y^2 + 10 z^2$ on a spherical shell with internal radius 0.1, outer radius 1 with the same boundary conditions. First, we define a sphericalShell mesh.

In [16]:
mesh_3D = uw.meshing.SphericalShell(radiusInner=0.1, 
                          radiusOuter=1, 
                          cellSize=0.1, qdegree=5)

We define our scalar $\phi$ on the mesh

In [17]:
phi = uw.discretisation.MeshVariable("r\phi", mesh_3D, 1, degree=3)

Define our poisson solver on the mesh

In [18]:
poisson_solver_3D = uw.systems.Poisson(mesh_3D, u_Field=phi)
poisson_solver_3D.constitutive_model = uw.systems.constitutive_models.DiffusionModel(mesh_3D.dim)

Define our source term $S(\vec{x}) = 6 + 10 x^2 + 10 y^2 + 10 z^2$

In [19]:
mesh_3D.X ## note that now, mesh_3D has three coordinates

Matrix([[N.x, N.y, N.z]])

In [20]:
x,y,z = mesh_3D.X ## access the symbolic repressentations of the mesh coordinates

source_fn = 6 + 10 * (x**2 + y**2 + z**2) ## source function
poisson_solver_3D.f = -source_fn 

Give the boundary conditions

In [21]:
poisson_solver_3D.add_dirichlet_bc((0.01,), "Lower", (0,))
poisson_solver_3D.add_dirichlet_bc((1,), "Upper", (0,))

In [22]:
poisson_solver_3D.constitutive_model.Parameters.diffusivity = 1
poisson_solver_3D.solve()

aList = [1 - i/3 for i in range(4)]
for a in aList:
    print(a)
    poisson_solver_3D.constitutive_model.Parameters.diffusivity = a + (1 - a)*k_fn
    poisson_solver_3D.solve(zero_init_guess=False)


  0 SNES Function norm 5.95949 
  1 SNES Function norm 0.000485061 
Nonlinear Poisson_2_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
1.0
  0 SNES Function norm 0.000485061 
  1 SNES Function norm 4.51262e-09 
Nonlinear Poisson_2_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
0.6666666666666667
  0 SNES Function norm 0.0934571 
  1 SNES Function norm 0.00709891 
  2 SNES Function norm 0.000688866 
  3 SNES Function norm 6.01285e-05 
  4 SNES Function norm 4.17179e-06 
Nonlinear Poisson_2_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 4
0.33333333333333337
  0 SNES Function norm 0.0664058 
  1 SNES Function norm 0.00852714 
  2 SNES Function norm 0.00129645 
  3 SNES Function norm 0.000168727 
  4 SNES Function norm 1.70409e-05 
  5 SNES Function norm 1.32688e-06 
Nonlinear Poisson_2_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 5
0.0
  0 SNES Function norm 0.0511853 
  1 SNES Function norm 0.00798184 
  2 SNES Function norm 0.00

In [23]:
x,y,z = mesh_3D.X
analytic_fn = x**2 + y**2 + z**2

In [24]:
with mesh_3D.access():
    mesh_analytic_soln = uw.function.evaluate(analytic_fn, mesh_3D.data, mesh_3D.N)
    mesh_numerical_soln = uw.function.evaluate(poisson_solver_3D.u.fn, mesh_3D.data, mesh_3D.N)
    

In [25]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh_3D.vtk("mesh_tmp.vtk")
    pvmesh = pv.read("mesh_tmp.vtk")

    with mesh_3D.access():
        pvmesh.point_data["T"] = mesh_analytic_soln
        pvmesh.point_data["T2"] = mesh_numerical_soln

    clipped = pvmesh.clip(origin=(0.001, 0.0, 0.0), normal=(1, 0, 0), invert=True)

    pl = pv.Plotter()

    pl.add_mesh(
        clipped,
        cmap="coolwarm",
        edge_color="Grey",
        show_edges=True,
        scalars="T",
        use_transparency=False,
        opacity=1.0,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    # pl.screenshot(filename="test.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_39276/3307009513.py:10: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='974...], height=500, min_width=500, sizing_mode='stretch_width')

In [26]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh_3D.vtk("mesh_tmp.vtk")
    pvmesh = pv.read("mesh_tmp.vtk")

    with mesh_3D.access():
        pvmesh.point_data["T"] = mesh_analytic_soln
        pvmesh.point_data["T2"] = mesh_numerical_soln

    clipped = pvmesh.clip(origin=(0.001, 0.0, 0.0), normal=(1, 0, 0), invert=True)

    pl = pv.Plotter()

    pl.add_mesh(
        clipped,
        cmap="coolwarm",
        edge_color="Grey",
        show_edges=True,
        scalars="T2",
        use_transparency=False,
        opacity=1.0,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    # pl.screenshot(filename="test.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_39276/2104744216.py:10: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='345...], height=500, min_width=500, sizing_mode='stretch_width')

In [27]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh_3D.vtk("mesh_tmp.vtk")
    pvmesh = pv.read("mesh_tmp.vtk")

    with mesh_3D.access():
        pvmesh.point_data["T"] = mesh_analytic_soln
        pvmesh.point_data["T2"] = mesh_numerical_soln
        pvmesh.point_data["diff"] = mesh_numerical_soln - mesh_analytic_soln
        

    clipped = pvmesh.clip(origin=(0.001, 0.0, 0.0), normal=(1, 0, 0), invert=True)

    pl = pv.Plotter()

    pl.add_mesh(
        clipped,
        cmap="coolwarm",
        edge_color="Grey",
        show_edges=True,
        scalars="diff",
        use_transparency=False,
        opacity=1.0,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    # pl.screenshot(filename="test.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_39276/4223488132.py:10: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='274...], height=500, min_width=500, sizing_mode='stretch_width')

In [28]:
with mesh_3D.access():
    maxDiff = float('-inf')
    for index in range(len(mesh_analytic_soln)):
        if ( abs(mesh_analytic_soln[index]-mesh_numerical_soln[index])/mesh_analytic_soln[index] > maxDiff ):
            maxDiff = abs(mesh_analytic_soln[index]-mesh_numerical_soln[index])/mesh_analytic_soln[index]
    print("Maximum percentage difference between analytic and numerical solution is", maxDiff*100)

Maximum percentage difference between analytic and numerical solution is 9.535019698571666
